In [1]:
import numpy as np
import skimage as ski
from skimage import io
import cv2 as cv

In [2]:
def show(img):
    cv.imshow('img', img)
    cv.waitKey(0)

In [3]:
img = cv.imread('training/001.tif', 0)

In [4]:
img.shape

(684, 489)

In [5]:
show(img)

In [5]:
ret, th1 = cv.threshold(img, 90, 255, cv.THRESH_BINARY)

In [7]:
show(th1)

In [20]:
np.unique(th1)

array([  0, 255], dtype=uint8)

In [6]:
thresh = th1

In [7]:
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
contours = max(contours, key=lambda x: cv.contourArea(x))
cv.drawContours(img, [contours], 0, (255,255,255), 2)

array([[28, 24, 27, ..., 32, 35, 31],
       [28, 25, 19, ..., 32, 33, 35],
       [25, 26, 24, ..., 35, 36, 36],
       ...,
       [23, 20, 20, ..., 37, 37, 35],
       [21, 22, 23, ..., 34, 33, 36],
       [21, 18, 18, ..., 37, 37, 32]], dtype=uint8)

In [10]:
show(img)

In [11]:
hull = cv.convexHull(contours)
cv.drawContours(img, [hull], -1, (255, 255, 255), 2)

array([[28, 24, 27, ..., 32, 35, 31],
       [28, 25, 19, ..., 32, 33, 35],
       [25, 26, 24, ..., 35, 36, 36],
       ...,
       [23, 20, 20, ..., 37, 37, 35],
       [21, 22, 23, ..., 34, 33, 36],
       [21, 18, 18, ..., 37, 37, 32]], dtype=uint8)

In [12]:
show(img)

In [8]:
hull = cv.convexHull(contours, returnPoints=False)
defects = cv.convexityDefects(contours, hull)

In [9]:
list_end = []
list_start = []

In [10]:
for i in range(defects.shape[0]):  # calculate the angle
    s, e, f, d = defects[i][0]
    start = tuple(contours[s][0])
    end = tuple(contours[e][0])
    far = tuple(contours[f][0])
    a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
    b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
    c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  #      cosine theorem
    if angle <= np.pi / 2:  # angle less than 90 degree, treat as fingers
        cv.circle(img, far, 4, [0, 0, 255], -1)
        list_start.append(start)
        list_end.append(end)

In [11]:
eps = 25

In [12]:
list_res = []
list_end_tmp = list_end.copy()
for first in list_start:
    for second in list_end:
        if np.sqrt((first[0] - second[0]) ** 2 + (first[1] - second[1]) ** 2) < eps:
            list_res.append(((first[0] + second[0]) // 2, (first[1] + second[1]) // 2))
            list_end_tmp.remove(second)
            break
    else:
        list_res.append(first)
if len(list_end_tmp) != 1:
    raise Exception('Not one')
else:
    list_res.append(list_end_tmp[0])

In [13]:
for point in list_res:
    cv.circle(img, point, 4, [0, 0, 255], -1)

In [20]:
show(img)

In [17]:
eps2 = 20

In [19]:
for i in range(defects.shape[0]):  # calculate the angle
    s, e, f, d = defects[i][0]
    start = tuple(contours[s][0])
    end = tuple(contours[e][0])
    far = tuple(contours[f][0])
    a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
    b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
    c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  #      cosine theorem
    if angle <= np.pi / 2:  # angle less than 90 degree, treat as fingers
        for point in list_res:
            if np.sqrt((point[0] - start[0]) ** 2 + (point[1] - start[1]) ** 2) < eps2:
                cv.line(img, point, far, [0, 0, 255], 3, -1)
            if np.sqrt((point[0] - end[0]) ** 2 + (point[1] - end[1]) ** 2) < eps2:
                cv.line(img, point, far, [0, 0, 255], 3, -1)